<a href="https://colab.research.google.com/github/hafidm93/gemini-idx-predict/blob/main/gemini_ML_idx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Dependencies

In [ ]:

!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install

In [ ]:
%pip install TA-Lib

In [ ]:
%pip install pandas yfinance scikit-learn numpy pytest-warnings markdown google-genai python_dotenv

## Collect data and Analyse

In [ ]:
import pandas as pd
import yfinance as yf
import talib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import numpy as np
import warnings
from dotenv import load_dotenv

load_dotenv()


warnings.filterwarnings('ignore')

# --- 1. Pengaturan dan Pengambilan Data (DIJAGA KONSISTEN) ---
def fetch_stock_data(ticker, start_date='2020-01-01', end_date=None):
    if end_date is None:
        end_date = pd.Timestamp.today().strftime('%Y-%m-%d')
    ticker_yf = ticker + ".JK"
    data = yf.download(ticker_yf, start=start_date, end=end_date, progress=False)

    if data.empty:
        print(f"⚠️ Gagal mengambil data untuk {ticker_yf}.")
        return None

    if isinstance(data.columns, pd.MultiIndex):
        data.columns = [col[0].lower().replace(' ', '_') for col in data.columns]
    else:
        data.columns = [col.lower().replace(' ', '_') for col in data.columns]

    if 'adj_close' in data.columns and 'close' in data.columns:
        data['close'] = data['adj_close']
        data = data.drop(columns=['adj_close'], errors='ignore')
    elif 'adj_close' in data.columns and 'close' not in data.columns:
        data = data.rename(columns={'adj_close': 'close'})

    required_cols = ['open', 'high', 'low', 'close', 'volume']
    if not all(col in data.columns for col in required_cols):
        missing_cols = [col for col in required_cols if col not in data.columns]
        print(f"⚠️ Data tidak lengkap. Kolom yang hilang: {missing_cols}")
        return None

    display(Markdown(data))
    return data

# --- 2. Perhitungan Indikator (Fitur X) dan Target (Y) - MODIFIKASI LENGKAP ---
def prepare_data_for_ml(data):
    """Menghitung fitur (X) dan variabel target (Y) termasuk fitur fundamental proxy."""
    if data is None:
        return None, None

    # --- 2.1. Feature Engineering: Indikator Teknikal (Momentum, Volatilitas) ---
    data['RSI'] = talib.RSI(data['close'], timeperiod=14)
    data['ADX'] = talib.ADX(data['high'], data['low'], data['close'], timeperiod=14)
    data['ATR'] = talib.ATR(data['high'], data['low'], data['close'], timeperiod=14)
    data['SMA_20'] = talib.SMA(data['close'], timeperiod=20)
    data['SMA_50'] = talib.SMA(data['close'], timeperiod=50)
    data['OBV'] = talib.OBV(data['close'], data['volume'])
    data['Volume_Ratio'] = data['volume'] / data['volume'].rolling(window=20).mean()
    data['MACD'], data['MACDSignal'], data['MACDHist'] = talib.MACD(data['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    data['BOP'] = talib.BOP(data['open'], data['high'], data['low'], data['close'])

    # New Technical Indicators (from previous turn)
    data['BBANDS_upper'], data['BBANDS_middle'], data['BBANDS_lower'] = talib.BBANDS(data['close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    data['STOCH_k'], data['STOCH_d'] = talib.STOCH(data['high'], data['low'], data['close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    data['CCI'] = talib.CCI(data['high'], data['low'], data['close'], timeperiod=14)

    # NEW Technical Indicators (for this turn)
    data['MFI'] = talib.MFI(data['high'], data['low'], data['close'], data['volume'], timeperiod=14)
    data['ULTOSC'] = talib.ULTOSC(data['high'], data['low'], data['close'], timeperiod1=7, timeperiod2=14, timeperiod3=28)


    # --- 2.2. Feature Engineering: PROXY FUNDAMENTAL & VALUASI ---

    # a. Aliran Bandar / Distribusi (Accumulation/Distribution Line)
    # Ini mengukur tekanan beli (akumulasi) atau jual (distribusi) secara kolektif.
    data['AD_Line'] = talib.AD(data['high'], data['low'], data['close'], data['volume'])
    # Tambahkan perubahan AD (Accumulation Change)
    data['AD_Change'] = data['AD_Line'].pct_change()

    # b. Proxy untuk Valuasi PBV & PE
    # Karena data Book Value per Share (BVS) dan Earnings per Share (EPS) tidak ada di yfinance,
    # kita tidak bisa menghitung PBV/PE yang sebenarnya.
    # Namun, kita bisa menggunakan Rasio Harga Jangka Panjang untuk Valuasi Relatif.

    # PBV/PE Banding Proxy: Harga Jarak Jauh (200 hari) vs Jarak Pendek (50 hari)
    data['SMA_200'] = talib.SMA(data['close'], timeperiod=200)
    # Rasio sebagai proxy valuasi: Jika harga saat ini jauh di atas SMA 200, mungkin overvalued secara historis
    data['Valuation_Ratio'] = data['close'] / data['SMA_200']

    # c. Proxy untuk EBITDA / Kinerja Laba
    # Gunakan Rate of Change (ROC) harga jangka menengah sebagai proxy kinerja yang tertunda.
    # Kenaikan harga jangka menengah sering kali disebabkan oleh rilis laba yang baik.
    data['ROC_60'] = talib.ROC(data['close'], timeperiod=60) # Price Rate of Change 60 hari

    # --- 2.3. Feature Engineering: Lagged Features (1 hari sebelumnya) ---
    lag_periods = [1]
    # Tambahkan fitur baru ke base_features
    base_features = [
        'RSI', 'ATR', 'Volume_Ratio', 'MACD', 'close',
        'AD_Change', 'Valuation_Ratio', 'ROC_60',
        'BBANDS_upper', 'BBANDS_middle', 'BBANDS_lower',
        'STOCH_k', 'STOCH_d', 'CCI',
        'MFI', 'ULTOSC'
    ]

    for feature in base_features:
        for lag in lag_periods:
            data[f'{feature}_lag_{lag}'] = data[feature].shift(lag)

    # --- 2.4. Target Variabel (Y) dan Cleaning ---
    data['Target'] = np.where(data['close'].shift(-1) > data['close'], 1, 0)

    # Pilih semua fitur yang tidak termasuk kolom dasar harga/volume dan target
    features = [col for col in data.columns if col not in ['open', 'high', 'low', 'close', 'volume', 'Target', 'MACDSignal', 'MACDHist', 'AD_Line']]

    # Convert infinite values to NaN so dropna can remove them
    data.replace([np.inf, -np.inf], np.nan, inplace=True)

    data = data.dropna()

    X = data[features]
    Y = data['Target']

    # Hapus baris terakhir
    X = X.iloc[:-1]
    Y = Y.iloc[:-1]

    return X, Y

# --- 3. Melatih dan Memprediksi dengan Random Forest (Dipertahankan) ---
def train_and_predict_rf(X, Y, ticker):
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=0.2, shuffle=False
    )

    model = RandomForestClassifier(
        n_estimators=500,
        random_state=42,
        class_weight='balanced',
        min_samples_split=5,
        max_depth=10
    )

    model.fit(X_train, Y_train)

    Y_pred = model.predict(X_test)
    accuracy = accuracy_score(Y_test, Y_pred)

    X_latest = X.iloc[[-1]]
    prediction = model.predict(X_latest)[0]

    cm = confusion_matrix(Y_test, Y_pred)
    TN, FP, FN, TP = cm.ravel()

    display(Markdown(
        f"<h3>{ticker}</h3>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<h4>Model Random Forest</h4>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<p>Akurasi Model: {accuracy*100:.2f}%</p>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<p>Matriks Kebingungan (Confusion Matrix):</p>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<p>  True Positives (Prediksi Naik & Benar): {TP}</p>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<p>  False Positives (Prediksi Naik, tapi Turun): {FP}</p>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<p>  True Negatives (Prediksi Turun & Benar): {TN}</p>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<p>  False Negatives (Prediksi Turun, tapi Naik): {FN}</p>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<p>Laporan Klasifikasi:</p>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<p>{classification_report(Y_test, Y_pred, target_names=['Turun (0)', 'Naik (1)'])}</p>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<p>PREDIKSI HARI TERAKHIR (RF) UNTUK {ticker}: Harga {ticker} diperkirakan akan **{'NAIK (1) ⬆️' if prediction == 1 else 'TURUN (0) ⬇️'}**.</p>",
        unsafe_allow_html=True

    ))
    

# --- 4. Fungsi Utama ---
def run_ml_prediction(ticker):
    data = fetch_stock_data(ticker)
    if data is None:
        return

    X, Y = prepare_data_for_ml(data)

    # Perlu setidaknya 200+ data karena ada perhitungan SMA 200
    if X is None or Y is None or X.shape[0] < 200:
        print("Data tidak cukup (minimal 200 hari) untuk melatih model dengan fitur fundamental proxy.")
        return

    train_and_predict_rf(X, Y, ticker)

## The Logic

In [ ]:
import pandas as pd
import yfinance as yf
import talib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import numpy as np
import warnings
from IPython.display import display, Markdown, Latex

warnings.filterwarnings('ignore')

# --- 1. Pengaturan dan Pengambilan Data (DIJAGA KONSISTEN) ---
def fetch_stock_data(ticker, start_date='2020-01-01', end_date=None):
    if end_date is None:
        end_date = pd.Timestamp.today().strftime('%Y-%m-%d')
    ticker_yf = ticker + ".JK"
    data = yf.download(ticker_yf, start=start_date, end=end_date, progress=False)

    if data.empty:
        print(f"⚠️ Gagal mengambil data untuk {ticker_yf}.")
        return None

    if isinstance(data.columns, pd.MultiIndex):
        data.columns = [col[0].lower().replace(' ', '_') for col in data.columns]
    else:
        data.columns = [col.lower().replace(' ', '_') for col in data.columns]

    if 'adj_close' in data.columns and 'close' in data.columns:
        data['close'] = data['adj_close']
        data = data.drop(columns=['adj_close'], errors='ignore')
    elif 'adj_close' in data.columns and 'close' not in data.columns:
        data = data.rename(columns={'adj_close': 'close'})

    required_cols = ['open', 'high', 'low', 'close', 'volume']
    if not all(col in data.columns for col in required_cols):
        missing_cols = [col for col in required_cols if col not in data.columns]
        print(f"⚠️ Data tidak lengkap. Kolom yang hilang: {missing_cols}")
        return None

    print(f"=======SUMMARY DATA FOR {ticker}")
    print(data)
    return data

# --- 2. Perhitungan Indikator (Fitur X) dan Target (Y) - MODIFIKASI LENGKAP ---
def prepare_data_for_ml(data):
    """Menghitung fitur (X) dan variabel target (Y) termasuk fitur fundamental proxy."""
    if data is None:
        return None, None

    # --- 2.1. Feature Engineering: Indikator Teknikal (Momentum, Volatilitas) ---
    data['RSI'] = talib.RSI(data['close'], timeperiod=14)
    data['ADX'] = talib.ADX(data['high'], data['low'], data['close'], timeperiod=14)
    data['ATR'] = talib.ATR(data['high'], data['low'], data['close'], timeperiod=14)
    data['SMA_20'] = talib.SMA(data['close'], timeperiod=20)
    data['SMA_50'] = talib.SMA(data['close'], timeperiod=50)
    data['OBV'] = talib.OBV(data['close'], data['volume'])
    data['Volume_Ratio'] = data['volume'] / data['volume'].rolling(window=20).mean()
    data['MACD'], data['MACDSignal'], data['MACDHist'] = talib.MACD(data['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    data['BOP'] = talib.BOP(data['open'], data['high'], data['low'], data['close'])

    # New Technical Indicators (from previous turn)
    data['BBANDS_upper'], data['BBANDS_middle'], data['BBANDS_lower'] = talib.BBANDS(data['close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    data['STOCH_k'], data['STOCH_d'] = talib.STOCH(data['high'], data['low'], data['close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    data['CCI'] = talib.CCI(data['high'], data['low'], data['close'], timeperiod=14)

    # NEW Technical Indicators (for this turn)
    data['MFI'] = talib.MFI(data['high'], data['low'], data['close'], data['volume'], timeperiod=14)
    data['ULTOSC'] = talib.ULTOSC(data['high'], data['low'], data['close'], timeperiod1=7, timeperiod2=14, timeperiod3=28)


    # --- 2.2. Feature Engineering: PROXY FUNDAMENTAL & VALUASI ---

    # a. Aliran Bandar / Distribusi (Accumulation/Distribution Line)
    # Ini mengukur tekanan beli (akumulasi) atau jual (distribusi) secara kolektif.
    data['AD_Line'] = talib.AD(data['high'], data['low'], data['close'], data['volume'])
    # Tambahkan perubahan AD (Accumulation Change)
    data['AD_Change'] = data['AD_Line'].pct_change()

    # b. Proxy untuk Valuasi PBV & PE
    # Karena data Book Value per Share (BVS) dan Earnings per Share (EPS) tidak ada di yfinance,
    # kita tidak bisa menghitung PBV/PE yang sebenarnya.
    # Namun, kita bisa menggunakan Rasio Harga Jangka Panjang untuk Valuasi Relatif.

    # PBV/PE Banding Proxy: Harga Jarak Jauh (200 hari) vs Jarak Pendek (50 hari)
    data['SMA_200'] = talib.SMA(data['close'], timeperiod=200)
    # Rasio sebagai proxy valuasi: Jika harga saat ini jauh di atas SMA 200, mungkin overvalued secara historis
    data['Valuation_Ratio'] = data['close'] / data['SMA_200']

    # c. Proxy untuk EBITDA / Kinerja Laba
    # Gunakan Rate of Change (ROC) harga jangka menengah sebagai proxy kinerja yang tertunda.
    # Kenaikan harga jangka menengah sering kali disebabkan oleh rilis laba yang baik.
    data['ROC_60'] = talib.ROC(data['close'], timeperiod=60) # Price Rate of Change 60 hari

    # --- 2.3. Feature Engineering: Lagged Features (1 hari sebelumnya) ---
    lag_periods = [1]
    # Tambahkan fitur baru ke base_features
    base_features = [
        'RSI', 'ATR', 'Volume_Ratio', 'MACD', 'close',
        'AD_Change', 'Valuation_Ratio', 'ROC_60',
        'BBANDS_upper', 'BBANDS_middle', 'BBANDS_lower',
        'STOCH_k', 'STOCH_d', 'CCI',
        'MFI', 'ULTOSC'
    ]

    for feature in base_features:
        for lag in lag_periods:
            data[f'{feature}_lag_{lag}'] = data[feature].shift(lag)

    # --- 2.4. Target Variabel (Y) dan Cleaning ---
    # Rename existing 1-day target
    data['Target_1_day'] = np.where(data['close'].shift(-1) > data['close'], 1, 0)

    # Create 3-month target (approx. 60 trading days)
    data['Target_3_month'] = np.where(data['close'].shift(-60) > data['close'], 1, 0)

    # Create 6-month target (approx. 120 trading days)
    data['Target_6_month'] = np.where(data['close'].shift(-120) > data['close'], 1, 0)

    # Select all features excluding base price/volume columns and target columns
    feature_cols = [col for col in data.columns if col not in [
        'open', 'high', 'low', 'close', 'volume',
        'Target_1_day', 'Target_3_month', 'Target_6_month',
        'MACDSignal', 'MACDHist', 'AD_Line'
    ]]

    # Convert infinite values to NaN so dropna can remove them
    data.replace([np.inf, -np.inf], np.nan, inplace=True)

    data = data.dropna()

    X = data[feature_cols]
    Y_all = data[['Target_1_day', 'Target_3_month', 'Target_6_month']]

    # Adjust slicing for future targets (remove last 120 rows where Target_6_month is NaN after dropna)
    # The previous dropna already handles this, but we slice explicitly to ensure targets are aligned with features.
    # However, since dropna was called after creating all targets, X and Y_all should already be aligned.
    # The last 120 values of Target_6_month would be NaN, and dropna removes them.
    # So, no additional slicing like X = X.iloc[:-120] or Y = Y.iloc[:-120] is explicitly needed here if dropna is effective.
    # Let's ensure no NaNs in targets due to shifting beyond available data before passing them out.
    # The .dropna() call after creating targets ensures that any rows with NaN in features or targets are removed.
    # Given that `shift(-120)` creates NaNs at the end, `dropna()` will remove those rows.

    return X, Y_all

# --- 3. Melatih dan Memprediksi dengan Random Forest (Dipertahankan) ---
def train_and_predict_rf(X, Y, ticker, target_name):
    # Ensure enough data for splitting and prediction
    if len(X) < 2:
        print(f"\nSkipping training for {ticker} for {target_name}: Not enough data after preprocessing.")
        return

    # Determine split point
    test_size = 0.2
    split_index = int(len(X) * (1 - test_size))

    X_train = X.iloc[:split_index]
    X_test = X.iloc[split_index:]
    Y_train = Y.iloc[:split_index]
    Y_test = Y.iloc[split_index:]

    # Ensure test sets are not empty
    if len(X_test) == 0 or len(Y_test) == 0:
        print(f"\nSkipping training for {ticker} for {target_name}: Test set is empty after splitting.")
        return

    model = RandomForestClassifier(
        n_estimators=500,
        random_state=42,
        class_weight='balanced',
        min_samples_split=5,
        max_depth=10
    )

    model.fit(X_train, Y_train)

    Y_pred = model.predict(X_test)
    accuracy = accuracy_score(Y_test, Y_pred)

    X_latest = X.iloc[[-1]]
    prediction = model.predict(X_latest)[0]

    # Fix: Ensure confusion_matrix always returns a 2x2 matrix by specifying labels
    cm = confusion_matrix(Y_test, Y_pred, labels=[0, 1])
    TN, FP, FN, TP = cm.ravel()

    display(Markdown(
        f"<h3>{ticker}</h3>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<h4>Model Random Forest</h4>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<p>Akurasi Model: {accuracy*100:.2f}%</p>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<p>Matriks Kebingungan (Confusion Matrix):</p>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<p>  True Positives (Prediksi Naik & Benar): {TP}</p>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<p>  False Positives (Prediksi Naik, tapi Turun): {FP}</p>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<p>  True Negatives (Prediksi Turun & Benar): {TN}</p>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<p>  False Negatives (Prediksi Turun, tapi Naik): {FN}</p>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<p>Laporan Klasifikasi:</p>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<p>{classification_report(Y_test, Y_pred, target_names=['Turun (0)', 'Naik (1)'])}</p>",
        unsafe_allow_html=True
    ))
    display(Markdown(
        f"<p>PREDIKSI HARI TERAKHIR (RF) UNTUK {ticker}: Harga {ticker} diperkirakan akan **{'NAIK (1) ⬆️' if prediction == 1 else 'TURUN (0) ⬇️'}**.</p>",
        unsafe_allow_html=True

    ))
    display(Markdown(
        f"NAIK (1) ⬆️" if prediction == 1 else "TURUN (0) ⬇️"
    ))


    prediction_text = "NAIK (1) ⬆️" if prediction == 1 else "TURUN (0) ⬇️"

    print("=======================================")

    accuraies = accuracy * 100
    res = {

        'ticker': ticker,
        'target_name': target_name,
        'accuracy': accuraies,
        'TP': TP,
        'FP': FP,
        'TN': TN,
        'FN': FN,
        'prediction': prediction_text

    }

    return res

# --- 4. Fungsi Utama ---
def run_ml_prediction(ticker):
    data = fetch_stock_data(ticker)
    if data is None:
        return

    X, Y_all = prepare_data_for_ml(data)

    # Perlu setidaknya data yang cukup setelah pembuatan fitur dan target
    # Mengingat SMA_200 dan shift(-120) untuk target, minimal data yang dibutuhkan lebih dari 200 hari + 120 hari.
    # Let's set a threshold of 350 days of data after all operations to be safe.
    if X is None or Y_all is None or X.shape[0] < 350:
        print(f"Data tidak cukup (minimal 350 hari setelah preprocessing) untuk melatih model dengan fitur fundamental proxy dan target multi-period untuk {ticker}.")
        return

    target_names = ['Target_1_day', 'Target_3_month', 'Target_6_month']

    response = {}
    for target_name in target_names:
        Y = Y_all[target_name]
        response[target_name] = train_and_predict_rf(X, Y, ticker, target_name)

    return response



## Get Ticker List

In [ ]:
PORTO = ['BDKR', 'BBTN', 'SAPX', 'BJBR', 'ACES', 'BFIN', 'BMTR', 'SPRE', 'BTEK', 'INPC', 'TMAS', 'ZINC', 'LEAD', 'GOTO', 'NFCX']
SCALPING = ['ARII', 'ARGO', 'BMHS', 'ISSP']
BANK = ['BBCA', 'BBRI', 'BBNI', 'BBTN', 'BMRI', 'BNLI', 'BBHI']

## Execute the Code

In [ ]:
import json

all_results = {}
# --- EKSEKUSI ---
for ticker in BANK:
    result = run_ml_prediction(ticker)

    all_results[ticker] = result

print("\n--- KONVERSI KE JSON ---")
# print(all_results)

print("JSON siap dikirim ke Gemini.")

display(Markdown(
    f"<p>JSON siap dikirim ke Gemini.</p>"
))

## Summizire the result using AI

In [ ]:
from google import genai
import os
apiKey = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=apiKey)


# --- MENGIRIM KE GEMINI ---

prompt_with_data = f"""
Tolong buat ringkasan prediksi saham berdasarkan data JSON berikut.
Fokus pada:
1.  **Ringkasan Prediksi NAIK vs TURUN**: Daftar Ticker yang diprediksi NAIK dan Ticker yang diprediksi TURUN, sebutkan juga tingkat akurasinya.
2.  **Kinerja Model**: Identifikasi Ticker dengan akurasi tertinggi dan terendah, jelaskan juga NAIK atau TURUN.

Buat dalam sebuah tabel sederhana dengan kolom-kolom berikut:
- Ticker
- Target
- Prediksi
- Akurasi

DATA HASIL PREDIKSI (JSON):
{all_results}
"""

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt_with_data,
)

display(Markdown(response.text))
